In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import netCDF4
import time
import xarray

In [ ]:
""" Read .nc file into pandas dataframe."""
#Runtime around 677 seconds using perf_counter.

tic = time.process_time()

nc_ibtracs = 'Allstorms.ibtracs_all.v03r10.nc'
path_to_nc = os.path.join('..','deconstruct_cyn',nc_ibtracs)

ibt_all_fields = xarray.open_dataset(path_to_nc).to_dataframe()
# name, storm_sn, time_record, landfall, 
# genesis_basin, season, 
# lat, lon, max_wind, min_pres, dist2land are the interesting variables

columns_we_want = ['storm_sn',
                   'name',
                   'source_time',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'source_lat',
                   'source_lon',
                   'source_wind',
                   'source_pres',
                   'dist2land']

ibt = ibt_all_fields[columns_we_want]

renamed_columns = ['storm_sn',
                   'name',
                   'time_recorded',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'lat',
                   'lon',
                   'max_wind',
                   'min_pres',
                   'dist2land']

columns = dict(zip(columns_we_want, renamed_columns))

toc = time.process_time()

elapsed = toc - tic
print(elapsed)
#ibt

In [ ]:
ibt2 = ibt.rename(index=str, columns = columns)

In [ ]:
ibt2.index.__dict__['_levels']

In [ ]:
ibt2.loc["10",:]

In [ ]:
ibt2

In [ ]:
ibt2.loc[pd.notna(ibt2["lat"]),:]

In [ ]:
""" Import landmask."""
#Elapsed time ~10 seconds.
tic = time.process_time()

nc_landmask = 'ETOPO1_Ice_g_gmt4.nc' #Land mask from ETOPO1 Global Relief Model
path_to_nc2 = os.path.join('..','deconstruct_cyn',nc_landmask)

landmask_all_fields = xarray.open_dataset(path_to_nc2).to_dataframe()

"""
columns_we_want = ['storm_sn',
                   'name',
                   'source_time',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'source_lat',
                   'source_lon',
                   'source_wind',
                   'source_pres',
                   'dist2land']

landmask = landmask_all_fields[columns_we_want]

columns = dict(zip(columns_we_want, renamed_columns))
"""

toc = time.process_time()

elapsed = toc - tic
print(elapsed)

In [ ]:
landmask_all_fields[landmask_all_fields.z > 0]

In [ ]:
landmask_all_fields.index.names

In [ ]:
landmask_all_fields['lon'] = pd.Series(landmask_all_fields.index._levels[1])

In [ ]:
land = landmask_all_fields[landmask_all_fields.z > 0]

### Creating a Boolean Mask for the Phillipines

Here we want to create a bounding box for the phillipines.  

In [ ]:
lm2 = landmask_all_fields.reset_index()

In [ ]:
eps = 1e-3

m1 = 1/5
m2 = -130/53
m3 = 27/56
m4 = -73/26

c1 = -5.38
c2 = +320.65
c3 = -55.61
c4 = +347.72

In [ ]:
lat_within_a = (18.1-eps <= lm2.lat) & (lm2.lat <= (19.2+eps))
lat_within_a.value_counts()

In [ ]:
lon_left_a = (lm2.lat - c1)/m1
lon_right_a = (lm2.lat - c2)/m2
lon_within_a = (lon_left_a <= lm2.lon) & (lm2.lon <= lon_right_a)
lon_within_a.value_counts()

In [ ]:
point_within_a = lat_within_a & lon_within_a
point_within_a.value_counts()

In [ ]:
lat_within_b = (6.2 <= lm2.lat) & (lm2.lat <= 18.1)
lat_within_b.value_counts()

In [ ]:
lon_left_b = (lm2.lat - c4)/m4
lon_right_b = (lm2.lat - c2)/m2
lon_within_b = (lon_left_b <= lm2.lon) & (lm2.lon <= lon_right_b)
lon_within_b.value_counts()

In [ ]:
point_within_b = lat_within_b & lon_within_b
point_within_b.value_counts()

In [ ]:
lat_within_gamma = (3.5 <= lm2.lat) & (lm2.lat <= 6.2)
lat_within_gamma.value_counts()

In [ ]:
lon_left_gamma = (lm2.lat - c4)/m4
lon_right_gamma = (lm2.lat - c3)/m3
lon_within_gamma = (lon_left_gamma <= lm2.lon) & (lm2.lon <= lon_right_gamma)
lon_within_gamma.value_counts()

In [ ]:
point_within_gamma = lat_within_gamma & lon_within_gamma
point_within_gamma.value_counts()



In [ ]:
point_within_phillipines_box = (point_within_a | point_within_b | point_within_gamma)
point_within_phillipines_box.value_counts()

In [ ]:
lm2['phil_box'] = point_within_phillipines_box

In [ ]:
lm2[lm2['phil_box']]

In [ ]:
lm2[(lm2['phil_box']) & (lm2['z'] > 0-eps)]